# Implementação do algoritimo completo, sem compromisso com  perfomance

## Imports

In [1]:
import os
import functools
import operator
import math
import random
import sys

import numpy as np

## Função match

![Match function](./img/match_function.png)

In [2]:
def cluster_matching_function(weight_matrix,
                              cluster_number,
                              element,
                              prototypes,
                              dissimilarity_matrices):
    """
        :params: weight_matrix: numpy array-like 
                    matriz K x P de pesos das matrizes de dissimilaridades por cluster
                cluster_number: int
                    Número do cluster em questão
                element: int
                    Índice do elemento (entre 0 e N-1)
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade

        :return: float

    """

    # Criando aliases compatíveis com as variáveis da fórmula
    k = cluster_number
    D = dissimilarity_matrices
    p = len(D)
    Gk = prototypes[k]
    l = weight_matrix

    dissimilarities_sum = np.array([dj[element, Gk].sum() for dj in D])

    return np.dot(l[k], dissimilarities_sum)

## Função objetivo

![Objetive function](./img/objective_function.png)

In [3]:
def objective_function(clusters_qtd,
                       elements_qtd,
                       adequacy_criterion,
                       m,
                       weight_matrix,
                       prototypes,
                       dissimilarity_matrices):
    """
        :params: clusters_qtd: int
                    Quantidade total de clusters
                elements_qtd: int
                    Quantidade de elementos da base de dados
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação 
                    de cada elemente a cada cluster
                m: int
                    Fator de ponderação do índice de adequação
                weight_matrix:
                     matriz K x P de pesos das matrizes de dissimilaridades por cluster
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade

        :return: float

    """

    u = np.power(adequacy_criterion, m) # Resolvendo a exponeciação de u de uma vez só
    l = weight_matrix
    D = dissimilarity_matrices
    K = clusters_qtd
    G = prototypes
    N = elements_qtd
    match = cluster_matching_function # Criando um alias para reduzir o nome da função de matching
  
    J = np.array([np.array([u[i, k] * match(l, k, i, G, D) for i in range(N)]).sum() 
          for k in range(K)])


    return J.sum()

## Protótipos

![Prototype function](./img/prototype_function.png)

In [4]:
def get_prototypes(elements_qtd,
                   q,
                   m,
                   s,
                   cluster_number,
                   adequacy_criterion,
                   dissimilarity_matrices,
                   weight_matrix):
    """
        :params:
                elements_qtd: int 
                    Quantidade de elementos da base de dados
                q: int
                    Quantidade de elementos protótipos
                m: int
                    Fator de ponderação do índice de adequação
                s: int
                    Fator de ponderação dos pesos das matrizes
                cluster_number: int
                    Quantidade total de clusters
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação 
                    de cada elemente a cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                weight_matrix: 
                     matriz K x P de pesos das matrizes de dissimilaridades por cluster

        :return: list

    """

    G = []
    k = cluster_number,
    D = dissimilarity_matrices
    u = np.power(adequacy_criterion, m)
    l = np.power(weight_matrix, s)
    N = elements_qtd
    p = len(D)
    
    p_sums = np.empty((p, N))
    #print("p_sums", p_sums.shape)
    
    for j in range(p):
        p_sums[j, :] = (D[j] * l[k,j]).sum(axis=1) 
    
    p_sums = p_sums.sum(axis=0)
    #print("p_sums somado", p_sums.shape)
    
    p_sums_rotten = u[:, k].flatten() * p_sums
    
    #print("u[:, k].shape", u[:, k].shape)
    #print("p_sums_pondered", p_sums_pondered.shape)
    
    #print(p_sums_pondered.shape)
    return p_sums_rotten.argsort()[-q:][::-1]
    
#     def dist(element):
#         """
#             Função auxiliar para cálculo da distância de um elemento qualquer 
#             em relação a todos os outros da base de dados, consideran as matrizes 
#             de dissimilaridade e 
#             o critério de adequação

#             return: (int, float)
#                 (element, soma das distâncias)
#         """
#         return element, sum([u[i, k] * sum([l[k, j] * D[j][element, i] for j in range(p)])
#                              for i in range(elements_qtd)])

#     while (len(G) < q):
#         # Calculando todas as distâncias dos elementos que ainda não estão em G
#         distances = [dist(i) for i in range(N) if i not in G]
#         # Obtendo o menor item de distância e separando entre o elemento e sua distância
#         element, _ = min(distances, key=operator.itemgetter(1))
#         G.append(element)

#     return G

## Matriz de relevâcia

![Funções de peso](./img/vector_weights_function.png)

In [5]:
def compute_relevance_weights(clusters_qtd,
                              dissimilarity_matrices,
                              prototypes,
                              elements_qtd,
                              adequacy_criterion,
                              m):
    """
        :params:
                clusters_qtd: int
                    Quantidade total de clusters
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                elements_qtd: int
                    Quantidade de elementos da base de dados
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação 
                    de cada elemente a cada cluster
                m: int
                    Fator de ponderação do índice de adequação

        :return: numpy array of shape K x P

    """

    D = dissimilarity_matrices
    P = len(D)
    Gk = prototypes
    K = clusters_qtd
    N = elements_qtd
    u = np.power(adequacy_criterion, m)
    l = np.zeros((K, P))

    def match(element, Dh, G):
        """
            Função auxiliar para cálculo de match entre um elemento 
            qualquer, os protótipos G de um cluster específico e uma matriz 
            de similaridade específica Dh.
        """

        #return sum([Dh[element, e] for e in G])
        return Dh[element, G].sum()

    for k in range(K):
        for j in range(P):
            # Calculado o somatório do numerador da equação à esquerda da igualdade
            weight_diss_sum1 = np.array([np.array([u[i, k] * match(i, D[h], Gk[k]) for i in range(N)]).sum()
                                for h in range(P)])

      
            # Calculado o somatório do denominador da equação à esquerda da igualdade
            weight_diss_sum2 = np.array([u[i, k] * match(i, D[j], Gk[k])
                                    for i in range(N)]).sum()

            # Executando a divisão da fração à esquerda da equação
            l[k, j] = math.pow(weight_diss_sum1.prod(), 1/P) / weight_diss_sum2

    return l

## Grau de pertinência

![Fórmula grau de pertinência](./img/membership_degree.png)

In [6]:
def compute_membership_degree(weight_matrix,
                              prototypes,
                              clusters_qtd,
                              dissimilarity_matrices,
                              elements_qtd,
                              m):
    """
        :params: weight_matrix: numpy array-like 
                    matriz K x P de pesos das matrizes de dissimilaridades por cluster
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                clusters_qtd: int
                    Quantidade total de clusters
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                elements_qtd: int
                    Quantidade de elementos da base de dados
                m: int
                    Fator de ponderação do índice de adequação

        :return: numpy array NxK

    """
        

    K = clusters_qtd
    G = prototypes
    D = dissimilarity_matrices
    l = weight_matrix
    P = len(D)
    N = elements_qtd
    u = np.zeros((N, K))
    
    match = cluster_matching_function # Criando um alias para reduzir o nome da função de matching

    def ratio(element, k, h):
        r = match(l, k, element, G, D) / match(l, h, element, G, D)
        return math.pow(r, 1/(m-1))

    for i in range(N):
        for k in range(K):
            outter_sum = np.array([ratio(i, k, h) for h in range(K)]).sum()
            u[i, k] = 1/outter_sum

    return u

## Carregando Matrizes

In [7]:
def carregar_matrizes(data_path = None):
    data_path = data_path or "./data"
    data_path = os.path.abspath(data_path)
    
    FAC_FILE = os.path.join(data_path, "mfeat-fac-dissimilarity.npy")
    FOU_FILE = os.path.join(data_path, "mfeat-fou-dissimilarity.npy")
    KAR_FILE = os.path.join(data_path, "mfeat-kar-dissimilarity.npy")

    fac_dis = np.load(FAC_FILE)
    fou_dis = np.load(FOU_FILE)
    kar_dis = np.load(KAR_FILE)
    
    return fac_dis, fou_dis, kar_dis

data_path = sys.argv[1] if len(sys.argv) == 2 else None
fac_dis, fou_dis, kar_dis = carregar_matrizes(data_path)

## Algoritmo completo
> Partitioning fuzzy K-medoids clustering algorithms with relevance weight for each dissimilarity matrix estimated locally

* Parametros: $K = 10; m = 1.6; T = 150; \epsilon = 10^{−10};$
* Devemos considerar a iniciarlizar do vetor de pesos como sendo 1, já que usamos a equação 9 (MFCMdd-RWL-P)

In [8]:
def random_prototypes(K, N, q, seed):
    elements = set(range(N))
    protos = []
    random.seed(seed)
    for k in range(K):
        protos.append(random.sample(elements, q))
        elements -= set(protos[-1])

    return protos

def assert_relevance_weights_prod_one(l):
    # l.shape == (K,P)
    prods_p = l.prod(axis=1)
    assert round(prods_p.sum()) == l.shape[0], f"O produto dos pesos de relevância não é igual a 1 ({prods_p.sum()})"

def assert_membership_degree_sum_one(u):
     # u.shape == (N,K)
    sums_k = u.sum(axis=1)
    assert round(sums_k.sum()) == u.shape[0], f"A soma dos pesos de relevância não é igual a 1 ({sums_k.sum()})"

def executar_treinamento(dissimilarity_matrices,
                       elements_qtd,
                       K=10,
                       m=1.6,
                       T=150,
                       epsilon=10e-10,
                       q=3, 
                       seed=13082020):

    D = dissimilarity_matrices
    N = elements_qtd
    P = len(D)

    last_lambda = np.ones((K, P))
#   last_lambda = np.random.dirichlet(np.ones(K), size=P)
    last_prototypes = random_prototypes(K, N, q, seed)
    last_membership_degree = None
    last_cost = None
    
    assert_relevance_weights_prod_one(last_lambda)

#     print("Passo 0")
#     print("Calculando matriz de adequação inicial (u0)")
    u0 = compute_membership_degree(weight_matrix=last_lambda,
                                   prototypes=last_prototypes,
                                   clusters_qtd=K,
                                   dissimilarity_matrices=dissimilarity_matrices,
                                   elements_qtd=N,
                                   m=m)
    
#     assert_membership_degree_sum_one(u0)

#     print("Calculando função de custo inicial (J0)")
    J0 = objective_function(clusters_qtd=K,
                            elements_qtd=N,
                            adequacy_criterion=u0,
                            m=m,
                            weight_matrix=last_lambda,
                            prototypes=last_prototypes,
                            dissimilarity_matrices=dissimilarity_matrices)
    
    last_membership_degree = u0
    last_cost = J0
    
    for t in range(1, T):
#         print(f"Passo {t}/{T}")
        
#         print(">> Calculando protótipos")
        new_prototypes = [get_prototypes(elements_qtd=N,
                                         q=q,
                                         m=m,
                                         s=1,
                                         cluster_number=k,
                                         adequacy_criterion=last_membership_degree,
                                         dissimilarity_matrices=D,
                                         weight_matrix=last_lambda) for k in range(K)]
        
        #print("new_prototypes.shape", new_prototypes)
        
#         print(">> Calculando matriz de relevâncias")
        new_lambda = compute_relevance_weights(clusters_qtd=K,
                                               dissimilarity_matrices=D,
                                               prototypes=new_prototypes,
                                               elements_qtd=N,
                                               adequacy_criterion=last_membership_degree,
                                               m=m)
        
#         assert_relevance_weights_prod_one(new_lambda)
    
#         print(">> Calculando grau de pertinência")
        new_degree = compute_membership_degree(weight_matrix=new_lambda,
                                               prototypes=new_prototypes,
                                               clusters_qtd=K,
                                               dissimilarity_matrices=dissimilarity_matrices,
                                               elements_qtd=N,
                                               m=m)
    
        
#         assert_membership_degree_sum_one(new_degree)

#         print(">> Calculando função objetivo")
        new_cost = objective_function(clusters_qtd=K,
                                      elements_qtd=N,
                                      adequacy_criterion=new_degree,
                                      m=m,
                                      weight_matrix=new_lambda,
                                      prototypes=new_prototypes,
                                      dissimilarity_matrices=dissimilarity_matrices)

        last_prototypes = new_prototypes
        last_lambda = new_lambda
        last_membership_degree = new_degree
#         print(">> Cost: ", new_cost)
        
        if abs(last_cost - new_cost) <= epsilon:
            last_cost = new_cost
            break
    
        last_cost = new_cost
        
    data = {
        "cost":last_cost,
        "membership_degree":last_membership_degree,
        "prototypes":last_prototypes,
        "weight_matrix":last_lambda,
        "times": t,
        "q": q,
        "K":K,
        "m":m,
        "seed": seed,
    }
    return data

## Executando 100x

In [9]:
from functools import partial
import multiprocessing as mp
import time

def executar_algoritmo_100x(dissimilarity_matrices, N, q=3):

    best = None
    
    seeds = [18082020 + i for i in range(101)]
    with mp.Pool() as p:
        results = []
        for seed in seeds:
            kwds = dict(q=q, seed = seed)
            r = p.apply_async(executar_treinamento, (dissimilarity_matrices, N), kwds)
            results.append(r)
            
        for i, r in enumerate(results):
            data = r.get()
            print(f"Execução {i+1}/100")
            print(">> Cost: ", data["cost"] )
            if (not best) or data["cost"] < best["cost"]:
                best = data 
        
            
    return best

melhor_resultado_fac = executar_algoritmo_100x([fac_dis], 2000, q=2)
melhor_resultado_fou = executar_algoritmo_100x([fou_dis], 2000, q=2)
melhor_resultado_kar = executar_algoritmo_100x([kar_dis], 2000, q=2)
melhor_resultado_todas = executar_algoritmo_100x([fac_dis, fou_dis, kar_dis], 2000, q=2)


Execução 1/100
>> Cost:  3859.3920163537523
Execução 2/100
>> Cost:  3979.353711930624
Execução 3/100
>> Cost:  4052.567391734742
Execução 4/100
>> Cost:  3876.17322319104
Execução 5/100
>> Cost:  4047.3045186431896
Execução 6/100
>> Cost:  4023.412888187279
Execução 7/100
>> Cost:  3987.5332763163005
Execução 8/100
>> Cost:  3787.2543301487976
Execução 9/100
>> Cost:  3891.272571076393
Execução 10/100
>> Cost:  3958.3853969874135
Execução 11/100
>> Cost:  3972.481827042848
Execução 12/100
>> Cost:  3877.0323828430455
Execução 13/100
>> Cost:  3811.04672352225
Execução 14/100
>> Cost:  3939.8717092373163
Execução 15/100
>> Cost:  3957.702833288195
Execução 16/100
>> Cost:  3797.8440998153633
Execução 17/100
>> Cost:  3932.092621123674
Execução 18/100
>> Cost:  3913.083911871279
Execução 19/100
>> Cost:  4023.6934131529883
Execução 20/100
>> Cost:  3814.8444305192716
Execução 21/100
>> Cost:  3850.071247669003
Execução 22/100
>> Cost:  3936.296601287829
Execução 23/100
>> Cost:  3842.10

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in double_scalars
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


Execução 64/100
>> Cost:  3771.784514423585
Execução 65/100
>> Cost:  3830.4901874173347
Execução 66/100
>> Cost:  nan
Execução 67/100
>> Cost:  4061.3325830317444
Execução 68/100
>> Cost:  3851.1822142430756
Execução 69/100
>> Cost:  4007.3002028809738
Execução 70/100
>> Cost:  3914.180306653844
Execução 71/100
>> Cost:  3966.0317615779786
Execução 72/100
>> Cost:  3911.1124742224156
Execução 73/100
>> Cost:  3833.2294381895676
Execução 74/100
>> Cost:  3960.852630060058
Execução 75/100
>> Cost:  3825.866698314764
Execução 76/100
>> Cost:  3913.8114090105373
Execução 77/100
>> Cost:  3828.7233059495393
Execução 78/100
>> Cost:  3821.196216537083
Execução 79/100
>> Cost:  3933.1174234749224
Execução 80/100
>> Cost:  4021.4618115052917
Execução 81/100
>> Cost:  3867.7769979828154
Execução 82/100
>> Cost:  3950.7582071072284
Execução 83/100
>> Cost:  3821.077964862724
Execução 84/100
>> Cost:  3846.629826348652
Execução 85/100
>> Cost:  4047.195341245923
Execução 86/100
>> Cost:  4022.03

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in double_scalars
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in double_scalars
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in double_scalars
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning

Execução 9/100
>> Cost:  nan
Execução 10/100
>> Cost:  2132.5234790745058
Execução 11/100
>> Cost:  2177.919651667071
Execução 12/100
>> Cost:  2168.7403275804177
Execução 13/100
>> Cost:  2119.2229184152834
Execução 14/100
>> Cost:  2125.0144974004397
Execução 15/100
>> Cost:  2086.63908198914
Execução 16/100
>> Cost:  2091.987591791848
Execução 17/100
>> Cost:  2097.6607452338367
Execução 18/100
>> Cost:  2149.0184355443635
Execução 19/100
>> Cost:  2158.646703204902
Execução 20/100
>> Cost:  2081.8214673706025
Execução 21/100
>> Cost:  2088.0446062343635
Execução 22/100
>> Cost:  2102.2584678619737
Execução 23/100
>> Cost:  2106.474517036893
Execução 24/100
>> Cost:  2092.611575634421
Execução 25/100
>> Cost:  2065.072991723312
Execução 26/100
>> Cost:  2080.1898941440922
Execução 27/100
>> Cost:  2080.2927550466566
Execução 28/100
>> Cost:  2113.740644068725
Execução 29/100
>> Cost:  2157.6793996104557
Execução 30/100
>> Cost:  2141.574314887281
Execução 31/100
>> Cost:  2102.26395

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in double_scalars
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in double_scalars
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in double_scalars
/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning

Execução 9/100
>> Cost:  nan
Execução 10/100
>> Cost:  1753.0608783508158
Execução 11/100
>> Cost:  1744.375061870458
Execução 12/100
>> Cost:  1715.9965197213266
Execução 13/100
>> Cost:  1699.7804341267038
Execução 14/100
>> Cost:  1696.1068708782113
Execução 15/100
>> Cost:  1704.7319231059168
Execução 16/100
>> Cost:  1728.3493154961884
Execução 17/100
>> Cost:  1712.1034752026944
Execução 18/100
>> Cost:  1713.3680250351226
Execução 19/100
>> Cost:  1708.2490190465514
Execução 20/100
>> Cost:  1712.087384170903
Execução 21/100
>> Cost:  1729.63484576169
Execução 22/100
>> Cost:  1759.335778613197
Execução 23/100
>> Cost:  1704.4877530190784
Execução 24/100
>> Cost:  1723.878700169882
Execução 25/100
>> Cost:  1721.003426664593
Execução 26/100
>> Cost:  1729.604411293109
Execução 27/100
>> Cost:  1722.3024187797728
Execução 28/100
>> Cost:  1695.020275320302
Execução 29/100
>> Cost:  1716.9910129081243
Execução 30/100
>> Cost:  1735.7894365827995
Execução 31/100
>> Cost:  1693.2753

Execução 95/100
>> Cost:  7296.31741588209
Execução 96/100
>> Cost:  7234.385713799807
Execução 97/100
>> Cost:  7148.100876785834
Execução 98/100
>> Cost:  7257.907972392087
Execução 99/100
>> Cost:  7394.026861084817
Execução 100/100
>> Cost:  7362.075039260294
Execução 101/100
>> Cost:  7307.675799629256


### Melhores resultados

In [10]:
print("Melhor custo pro fac", melhor_resultado_fac["cost"])
print("Melhor custo pro fou", melhor_resultado_fou["cost"])
print("Melhor custo pro kar", melhor_resultado_kar["cost"])
print("Melhor custo com as três matrizes", melhor_resultado_todas["cost"])

Melhor custo pro fac 3757.734779046913
Melhor custo pro fou 2047.1053313647592
Melhor custo pro kar 1678.2024241276338
Melhor custo com as três matrizes 7148.100876785834


### Exportando melhores resultados

In [11]:
import pickle
 
def export_best_result(data, file_name):
    with open(file_name, "wb") as f:
        pickle.dump(data, f)

export_best_result(melhor_resultado_fac, "data/melhor_resultado_fac.pickle")
export_best_result(melhor_resultado_fou, "data/melhor_resultado_fou.pickle")
export_best_result(melhor_resultado_kar, "data/melhor_resultado_kar.pickle")
export_best_result(melhor_resultado_todas, "data/melhor_resultado_todas.pickle")

## Partições hard

In [12]:
def get_hard_patitions(membership_degree):
    """
        membership_degree: numpy array of shape N x K
    """
    u = membership_degree
    members = []
    K = membership_degree.shape[1]
    
    # Obtendo o índice do grupo em que cada elemento possui maior valor de pertencimento
    element_index_membership = u.argsort(axis=1)[:, 0]
    
    for k in range(K):
        # Para cada grupo k, extrai quais dos elementos possui maior grau de pertencimento para ele
        memb = np.where(element_index_membership == k)[0]
        members.append(memb)
        
    return members

membership_degree = melhor_resultado_todas["membership_degree"]
hard_members = get_hard_patitions(membership_degree)

for group_number, group in enumerate(hard_members):
    print(group_number, group, len(group))

print("Total de elementos somados:",  sum([len(g) for g in hard_members]))

NameError: name 'clustering_data' is not defined

## Funções de validação do cluster

In [ ]:
def calc_partition_coefficient(membership_degree):
    n = membership_degree.shape[0]
    membership_degree = np.power(membership_degree, 2)
    return membership_degree.sum(axis = 1).sum()/n

def calc_modified_partition_coefficient(membership_degree):
    c = membership_degree.shape[1]
    vpc = calc_partition_coefficient(membership_degree)
    return 1 - (c/(c-1))*(1-vpc)

def calc_partition_entropy(membership_degree):
    n = membership_degree.shape[0]
    membership_degree = np.log10(membership_degree) * membership_degree
    return -membership_degree.sum(axis = 1).sum()/n

partition_coeficient = calc_modified_partition_coefficient(membership_degree)
partition_entropy = calc_partition_entropy(membership_degree)

print("Modified partition coefficient:", partition_coeficient)
print("Parition Entropy:", partition_entropy)
